# Dependencies

In [20]:
import nrrd
import slicerio as si
import os
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
import med_dataloader as mdl
import tensorflow as tf
import numpy as np

# Segmentations Check

In [19]:
dataset_dir = os.path.join(os.getcwd(), "Dataset")
segmentation_dir = os.path.join(dataset_dir , "Segmentation") #mettere dataset nella working directory
segmentation_checked_dir = os.path.join(dataset_dir , "Segmentation_checked")

try:
    os.mkdir(segmentation_checked_dir)
except:
    pass


## Check

In [21]:

input_filenames = os.listdir(segmentation_dir)
segment_list = [("LL", 5), ("SP", 3), ("TO", 4), ("UL", 1), ("HE", 6), ("HP", 2)]

seg = 0

for segmentation in input_filenames:
    input_filename = os.path.join(segmentation_dir, input_filenames[seg])
    output_filename = os.path.join(segmentation_checked_dir, input_filenames[seg])
    seg = seg + 1
    voxels, header = nrrd.read(input_filename)
    segmentation_info = si.read_segmentation_info(input_filename)
    output_voxels, output_header = si.extract_segments(voxels, header, segmentation_info, segment_list)
    output_voxels = output_voxels.astype("uint8") #necessario per avere un nnrd in output con type 'uint8' anzichè 'float64' non gestito da med_dataloader
    nrrd.write(output_filename, output_voxels, output_header)


## Mac os Compatibility

In [8]:
try:
    os.system("rm" , os.path.join(segmentation_checked_dir, ".DS_Store"))
except BaseException:
    pass

In [22]:
print(len(input_filenames))
output_filenames = os.listdir(segmentation_checked_dir)
print(len(output_filenames))

460
460


## Control Plotting
* Display a comparison between original segmentation and segmentation checked

In [ ]:
seg = 0

for segmentation in  input_filenames:
    input_filename = os.path.join(segmentation_dir, input_filenames[seg])
    output_filename = os.path.join(segmentation_checked_dir, input_filenames[seg])
    seg = seg + 1
    voxels, header = nrrd.read(input_filename)
    output_voxels, header = nrrd.read(output_filename)

    fig = plt.figure(figsize = [45, 15])
    plt.subplot(1, 2, 1)
    plt.imshow(voxels[:,:,0], cmap="magma")
    plt.subplot(1, 2, 2)
    plt.imshow(output_voxels[:,:,0], cmap="magma")
    plt.show(fig)
    

# Dataloader

In [23]:
NUM_CLASSES = 7

In [24]:
mdl.generate_dataset(data_dir= os.path.join(dataset_dir),
                        imgA_label="Images",
                        imgB_label="Segmentation_checked",
                        input_size=256,
                        is_B_categorical=True,
                        num_classes = NUM_CLASSES,
                        norm_boundsA=[0,255], #normalizza tra 0 e 1; bisogna mettere lower_bound,upper_bound dell'immagine corrente!!! (non i limiti che si vogliono raggiungere)
                        norm_boundsB=None,
                        )

Caching decoded images in c:\Users\matte\Desktop\Network-Vocal_Tract_Segmentation\Dataset_TF\Images.cache...
460/460
Cached decoded images in c:\Users\matte\Desktop\Network-Vocal_Tract_Segmentation\Dataset_TF\Images.cache.
Caching decoded images in c:\Users\matte\Desktop\Network-Vocal_Tract_Segmentation\Dataset_TF\Segmentation_checked.cache...
460/460
Cached decoded images in c:\Users\matte\Desktop\Network-Vocal_Tract_Segmentation\Dataset_TF\Segmentation_checked.cache.


# Utils

In [ ]:

max = 0
min = 255
for batch in train_ds.take(93): #meno hc
    volume_batch, label_batch = batch
    for i in range(4): #meno hc
        max_start = np.max(volume_batch[i,:,:])
        min_start = np.min(volume_batch[i,:,:])
        if max_start > max:
            max = max_start
        if min_start < min:
            min = min_start

#print(np.shape(volume_batch))
print(max, min)